# Generate set of administrative regions to use in exposure grid
- The idea here is to get a globally comprehensive set of administrative boundaries at the `adm1` level or higher if the `adm1` level is not available
- Use `adm1` if available and `adm0` if not

In [1]:
import geopandas as gpd
import pandas as pd
from sliiders import settings as sset

#### Read full `adm0` and `adm1` sets

In [ ]:
adm0 = gpd.read_file(sset.PATH_GADM, layer="level0").set_crs(epsg=4326)
adm1 = gpd.read_file(sset.PATH_GADM, layer="level1").set_crs(epsg=4326)

# Select columns
adm0 = adm0[["GID_0", "geometry"]].copy()
adm1 = adm1[["GID_0", "GID_1", "geometry"]].copy()

#### Exclude non-territory ISOs
- `ATA`: Antarctica
- `XCA`: Caspian Sea

In [4]:
sset.EXCLUDED_ISOS

['ATA', 'XCA']

In [ ]:
# Filter out excluded ISOs
adm0 = adm0[~adm0["GID_0"].isin(sset.EXCLUDED_ISOS)].copy()
adm1 = adm1[~adm1["GID_0"].isin(sset.EXCLUDED_ISOS)].copy()

#### Select only countries in `adm0` that do not appear in the `adm1` set, to be included as intermediate `adm1` regions

In [ ]:
adm0_as_adm1 = adm0[~adm0["GID_0"].isin(adm1["GID_0"].unique())].copy()

#### Concatenate all `adm1` regions

In [ ]:
full = pd.concat([adm0_as_adm1, adm1], ignore_index=True)

#### Assign a comprehensive set of unique IDs
- `GID_1` (`adm1`) for actual `adm1` regions
- `adm0` ID for `adm0_as_adm1` regions

In [ ]:
full["UID"] = full["GID_1"].fillna(full["GID_0"])

## Generate global Voronoi shapes for `adm1` regions
- Here, `adm1` includes `adm0` for which there are no `adm1` regions

In [ ]:
adm1 = spatial.get_voronoi_regions(full)
adm1["ISO"] = adm1["UID"].str.split(".").str[0]
countries = adm1.dissolve("ISO", as_index=False).drop(columns=["UID"])

## Assign ISO to GTSM / CIAM points
- assign based on country Voronoi
- countries are defined by impact regions

In [ ]:
stations = spatial.match_stations_to_countries(countries, sset.PATH_SEG_CENTROIDS)

## Generate ISO-level point-voronoi from CIAM points

In [ ]:
vor_gdf = spatial.get_stations_by_iso_voronoi(stations)

## Get CIAM coastline by country

In [ ]:
coastlines = spatial.get_coastlines_by_iso(
    gpd.read_file(sset.PATH_CIAM_COASTLINES), adm1
)

## Get coast-seg-by-CIAM point

In [ ]:
coastal_segs = spatial.get_coastal_segments_by_ciam_site(vor_gdf, coastlines)

## Overlap coastline vor with adm1 vor to get spatially comprehensive seg_adm1.

In [ ]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)
    all_overlays, ciam_polys = spatial.generate_voronoi_from_segments(
        coastal_segs,
        adm1,
        "UID",
        "seg_adm",
        "adm1",
    )

In [ ]:
sset.PATH_CIAM_ADM1_VORONOI_INTERSECTIONS.parent.mkdir(exist_ok=False)

all_overlays.to_parquet(sset.PATH_CIAM_ADM1_VORONOI_INTERSECTIONS, index=False)
all_overlays.to_file(sset.PATH_CIAM_ADM1_VORONOI_INTERSECTIONS_SHP)

ciam_polys.to_parquet(sset.PATH_CIAM_POLYS, index=False)